# initial setup

In [1]:
options nonotes nosource nosource2 errors=0;

In [ ]:
proc lua;
submit;

endsubmit;
run;

# Lua 101

### There is only 1 data structure: TABLE

In [3]:
proc lua;
submit;

  -- declare a table
  t = {}

endsubmit;
run;

### Table as Array

In [12]:
proc lua;
submit;

  -- declare a table 
  local shoppinglist = {'milk', 'flour', 'eggs', 'sugar'}
  print(shoppinglist)
  
  --[[ like SAS DATA step arrays, 
    arrays in Lua start at index 1 by default
  --]]
  local drink = shoppinglist[1]
  print(drink)
  
  --iterate
  for i, v in ipairs(shoppinglist) do
      print(i,v)
  end

endsubmit;
run;

### Table as Dictionary (Hash)

In [14]:
proc lua;
submit;

    local band = {
      vocals='Robert Plant',
      guitar='Jimmy Page',
      bass='John Paul Jones',
      drummer='John Bonham'
    }
    
    --two ways for key-value index
    print(band["bass"])
    print(band.guitar)
    
    --iterate
    for i, item in pairs(band) do
        print(item,"plays",i)
    end
    
endsubmit;
run;

### More on ipairs and pairs

In [19]:
proc lua;
submit;

   t = {fred='one',[0]=1; 10,20,30,40}
   
   print(" ==pairs iterates through all the key/value combinations ")
   
   for k,v in pairs(t) do
       print(k,v)
   end
    
    print(" ==ipairs iterates through the index of the array")
    
    for i,v in ipairs(t) do 
       print(i,v) 
    end

endsubmit;
run;

# Wraper

## Task 1: Does a Dataset Exist?  

### \#1 Data Step

In [23]:
data _null_;
    fname="tmp";
    rc=filename(fname,"class.sas7dat");
    
    if fexist(fname) then do;
       put "haha";
    end;
    
    else put "wow";
    rc=filename(fname);
run;

### \#2 Macro

In [22]:
%macro check_it(ds);
    %if %sysfunc(exist(&ds)) %then %do;	
        %put &ds exists. Good!;
    %end;
    
    %else %do;
        %put ERROR: &ds does not exist. Bad!;
        %abort cancel;
    %end;
%mend check_it;

%check_it(sashelp.class)

### \#3 Lua

In [30]:
proc lua ;
submit; 

if sas.exists("sashelp.class") then
    print ("haha")
    
    sas.submit [[
      data class;
        set sashelp.class;
        a = 1;
      run;

      proc print data=class(obs=2);
      run;
     ]]
 
else print "clash.."
end
 
endsubmit;
run;


# Benefits of Proc Lua:  
- It looks elegant. So it’s much preferable from aesthetic point of view.  
- The programming elements are rich enough inside the Proc Lua wrapper: you can run Lua scripts, and also SAS Data Steps and Procedures.  
- Easy to debug.

## Task 2: Dynamic Programming

### \#1 SAS Macro Array

In [48]:
proc sort data=sashelp.class (keep=sex) nodupkey out=gender; 
    by sex; 
run; 

data _null_; 
    set gender end=eof; 
    i+1; 
    II=left(put(i,2.)); 
    call symputx('sex'||II,sex); 
    if eof then call symputx('n',II); 
run; 

%macro doit; 
%do i=1 %to &n; 
    proc print data=sashelp.class; 
        where sex="&&sex&i"; 
    run; 
%end; 
%mend; 

%doit

### \#2 Lua

In [50]:
proc lua; 
submit;

  local dsid =sas.open("sashelp.class")

  local sex = {}
  local hash = {}
  local res = {}

  for row in sas.rows(dsid) do
    sex[#sex+1] = row.sex
    for _,v in ipairs(sex) do
        if (not hash[v]) then
            res[#res+1] = v
            hash[v] = true
        end
    end
  end

  sas.close(dsid)

  for i,sex in ipairs(res) do
           sas.submit[[
            proc print data=sashelp.class;
                where sex="@sex@";
            run;  
            ]]
end

endsubmit;
run;


# Benefits of Proc Lua  
- It can iterate with SAS datasets 
- Advanced data structure like dictionary can be applied to SAS programming  
- Advance (and REAL) For Loop can be used  

# SAS Options Management

- SAS options are global.   
- It’s designed in this way for good purpose.   
- But sometimes you just want to evoke a SAS option but are too lazy to close it at the end.   

In [47]:
proc lua;
    submit;
        sas.submit[[
            options obs=1;
            proc print data=sashelp.class;
            run;
        ]]
    endsubmit;
run;


proc print data=sashelp.class(obs =2);
run;


# Reading JSON File!!  
### http://regex.info/blog/lua/json  

In [58]:
proc lua ; 
submit;

    JSON = (loadfile "JSON.lua")() -- one-time load of the routines

    local raw_json_text = '{"Hello":["lunajson",1.5]}'
    local lua_value = JSON:decode(raw_json_text) -- decode example
    local pretty_json_text = JSON:encode_pretty(lua_value) -- "pretty printed" version

    print(pretty_json_text)

endsubmit;
run;


# Machine Learning!  
##  http://lonelydeveloper.org/workspace/projects/misc/nbc.lua 

In [59]:
proc lua infile="nbc" restart;
    submit;

    nbc = require('nbc')

    training_set = {
  { "sunny", "hot", "high", "weak", 0 },
  { "sunny", "hot", "high", "strong", 0 },
  { "cloudy", "hot", "high", "weak", 1 },
  { "rainy", "temperate", "high", "weak", 1 },
  { "rainy", "cold", "normal", "weak", 1 },
  { "rainy", "cold", "normal", "strong", 0 },
  { "cloudy", "cold", "normal", "strong", 1 },
  { "sunny", "temperate", "high", "weak", 0 },
  { "sunny", "cold", "normal", "weak", 1 },
  { "rainy", "temperate", "normal", "weak", 1 },
  { "sunny", "temperate", "normal", "strong", 1 },
  { "cloudy", "temperate", "high", "strong", 1 },
  { "cloudy", "hot", "normal", "weak", 1 },
  { "rainy", "temperate", "high", "strong", 0 }
     }

    domains = {
      { "sunny", "cloudy", "rainy" },
      { "hot", "temperate", "cold" },
      { "high", "normal" },
      { "weak", "strong" },
      { 0, 1 }
    }

    print( nbc.NaiveBayesClassifier(training_set, domains, {"sunny", "hot", "high", "weak"}) )

    print('--')

    print( nbc.NaiveBayesClassifier(training_set, domains, {"rainy", "temperate", "high", "weak"}) )

    endsubmit;
run;


# OOP Programming  
### https://medium.com/@yoshihirofukiya/proc-lua%E3%82%92%E5%8B%89%E5%BC%B7%E3%81%97%E3%81%A6%E3%81%BF%E3%81%A6%E3%81%AE%E6%84%9F%E6%83%B3%E3%81%A8proc-lua%E3%82%92%E3%82%AA%E3%83%96%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E6%8C%87%E5%90%91%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%82%B0%E3%81%AE%E8%A9%A6%E3%81%BF-e83929d59898#.4jjrmwkei  

In [71]:
proc lua;
  submit;
    
    datasetclass={}
    datasetclass.nobs = function(self)
      local dsid = sas.open(self.ds)
      local nobs = sas.nobs(dsid)
      sas.close(dsid)
      return nobs
    end
    
    datasetclass.nvars = function(self)
      local dsid = sas.open(self.ds)
      local nvars = sas.nvars(dsid)
      sas.close(dsid)
      return nvars
    end

    datasetclass.label = function(self)
      local dsid = sas.open(self.ds)
      local nvars = sas.attr(dsid,"label")
      sas.close(dsid)
      return nvars
    end

    datasetclass.meta = function(self)
      local i = 1
      local meta={}
      local dsid = sas.open(self.ds)
      for i = 1, sas.nvars(dsid) do
        meta[i]=table.tostring(sas.varinfo(dsid,i))
      end
      sas.close(dsid)
      return meta
    end
    
    -- new method (constructor)
    datasetclass.new = function(_ds)
      local obj = {}
      obj.ds = _ds
      setmetatable(obj,{__index=datasetclass})
      return obj
    end    
    
    local sampleds1=datasetclass.new("sashelp.class")
    print("nobs=",sampleds1:nobs())
    print("nvars=",sampleds1:nvars())
    print("label=",sampleds1:label())
    
  endsubmit;
run;


# Final Thoughts  

## SAS is not “A” programming language.   

## it’s a collection of multiple programming languages, which includes  
- Data Step  
- DS2  
- Macro  
- SQL  
- IML  
- Groovy, and  
- Lua

## Proc Lua offers a totally different programming style and flavor like  
- a data structure   
- for loop   
- external packages support  
- object-oriented programming, and   
- other modern scripting language features  